In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
# Set up Chrome with options
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")  # Avoid bot detection
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")  # Fake user-agent
options.add_argument("--incognito")  # Open in incognito mode

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)



In [30]:
def weworkremotelyJobs(title): 
    url = f"https://weworkremotely.com/remote-jobs/search?search_uuid=&term={title}&button=&sort=past_week" 
    driver.get(url) 
    wait = WebDriverWait(driver, 10)  
     
    try: 
        jobs_list = wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="category-17"]/article/ul'))) 
        job_items = jobs_list.find_elements(By.TAG_NAME, 'li')  
        print(f"Total jobs found: {len(job_items)}") 
        jobs = [] 
     
        for job in job_items:  
            try: 
                if "view-all" in job.get_attribute('class'): 
                    continue 
                
                href = job.find_element(By.TAG_NAME, 'a').get_attribute('href') 
                title = job.find_element(By.CLASS_NAME, 'title').text 
                company_info = job.find_elements(By.CLASS_NAME, 'company')
                company = company_info[0].text if len(company_info) > 0 else "Unknown"
                job_type = company_info[1].text if len(company_info) > 1 else "Not specified"
                # Handle missing location
                try:
                    location = job.find_element(By.CLASS_NAME, 'region.company').text 
                except:
                    location = "Not specified"
                
                # Handle missing posted date
                try:
                    posted_elements = job.find_elements(By.CLASS_NAME, 'listing-date__date')
                    posted = posted_elements[0].text if posted_elements else "Not provided"
                except:
                    posted = "Not provided"

                # Handle missing image
                try:
                    img = job.find_element(By.CLASS_NAME, 'flag-logo').get_attribute('style') 
                    start = img.find('url("')+5  # Find start of URL
                    img = img[start:len(img)-3]
                except:
                    img = "No image"

                job_as_JSON = { 
                    "title": title, 
                    "company": company, 
                    "location": location, 
                    "posted": posted, 
                    "href": href, 
                    "img": img,
                    "type":job_type
                } 
                jobs.append(job_as_JSON) 
                print(f"Title: {title}\nCompany: {company}\nLocation: {location}\nPosted: {posted}\nLink: {href}\nImage: {img}\n") 
 
            except Exception as e: 
                print(f"Error extracting job details: {e}") 

        with open("weworkremotely_jobs.json", "w", encoding="utf-8") as f: 
            json.dump(jobs, f, indent=4, ensure_ascii=False) 

    except Exception as e: 
        print("Job listings not found or error:", e) 

weworkremotelyJobs('Engineer') 

Total jobs found: 6
Title: Software Engineer (Python), Data Platform
Company: Doximity
Location: USA Only
Posted: 2d
Link: https://weworkremotely.com/company/doximity
Image: No image

Title: Senior Frontend Engineer (React, TypeScript)
Company: Paperpile
Location: Anywhere in the World
Posted: 4d
Link: https://weworkremotely.com/company/paperpile
Image: https://we-work-remotely.imgix.net/logos/0162/2481/logo.gif?ixlib=rails-4.0.0&w=50&h=50&dpr=2&fit=fill&auto=compress

Title: Senior Mobile Engineer (iOS and Android)
Company: Paperpile
Location: Anywhere in the World
Posted: 4d
Link: https://weworkremotely.com/company/paperpile
Image: https://we-work-remotely.imgix.net/logos/0162/2481/logo.gif?ixlib=rails-4.0.0&w=50&h=50&dpr=2&fit=fill&auto=compress

Title: Senior Software Engineer, Web
Company: Speechify Inc
Location: Anywhere in the World
Posted: 6d
Link: https://weworkremotely.com/company/speechify-inc
Image: No image

Title: Senior Frontend Engineer
Company: Chameleon
Location: Othe

In [ ]:
# url = f"https://weworkremotely.com/remote-jobs/search?search_uuid=&term={title}&button=&sort=past_week" #✅✅✅
# url = f"https://remoteok.com/remote-engineer-jobs" # scrape the filter options
# url = f"https://remotive.io/remote-jobs?query={title}" #✅✅✅


In [46]:
import time
def remotiveJobs(title):
    url = f"https://remotive.io/remote-jobs?query={title}" 
    driver.get(url) 
    wait = WebDriverWait(driver, 10)  
    
    try: 
        # Wait for the job list to load completely
        wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="hits"]/ul/div')))
        
        job_items = driver.find_elements(By.XPATH, '//*[@id="hits"]/ul/div')
        print(f"Total jobs found: {len(job_items)}") 
        jobs = [] 

        for job in job_items:  
            try: 
                if "view-all" in job.get_attribute('class'): 
                    continue 

                href = job.find_element(By.TAG_NAME, 'a').get_attribute('href') 
                title_element = job.find_elements(By.CLASS_NAME, 'remotive-bold')

                title = title_element[0].text if len(title_element) > 0 else "Unknown"
                company = title_element[2].text if len(title_element) > 1 else "Not specified"

                location = "Specified in Job description"
                job_type = "Specified in Job description"

                try:
                    posted_elements = job.find_elements(By.XPATH, './/span[contains(@class, "text-gray-500")]')
                    posted = posted_elements[0].text if posted_elements else "Not provided"
                except:
                    posted = "Not provided"

                try:
                    # Wait for the image to be visible, and then get the src attribute
                    img_elements = job.find_elements(By.TAG_NAME, 'img')
                    img = img_elements[0].get_attribute('src') if img_elements else "No Image"
                except Exception:
                    img = "No Image"

                job_as_JSON = { 
                    "title": title, 
                    "company": company, 
                    "location": location, 
                    "posted": posted, 
                    "href": href, 
                    "img": img,
                    "type": job_type,
                } 
                jobs.append(job_as_JSON) 
                print(f"Title: {title}\nCompany: {company}\nPosted: {posted}\nLink: {href}\n") 

            except Exception as e: 
                print(f"Error extracting job details: {e}") 

        with open("remotive_jobs.json", "w", encoding="utf-8") as f: 
            json.dump(jobs, f, indent=4, ensure_ascii=False) 

    except Exception as e: 
        print("Job listings not found or error:", e)

remotiveJobs('Engineer')

Total jobs found: 25
Title: Senior Frontend Developer
Company: Mimo GmbH
Posted: Not provided
Link: https://remotive.com/remote-jobs/software-dev/senior-frontend-developer-1968223

Title: Senior Backend Developer
Company: Mimo GmbH
Posted: Not provided
Link: https://remotive.com/remote-jobs/software-dev/senior-backend-developer-1968220

Title: Senior Independent Software Developer
Company: A.Team
Posted: Not provided
Link: https://remotive.com/remote-jobs/software-dev/senior-independent-software-developer-1919265

Title: Senior Independent UX/UI Designer
Company: A.Team
Posted: Not provided
Link: https://remotive.com/remote-jobs/design/senior-independent-ux-ui-designer-1919266

Title: Senior DevOps Engineer (AWS)
Company: Proxify
Posted: Not provided
Link: https://remotive.com/remote-jobs/devops/senior-devops-engineer-aws-1359476

Title: Software Engineer - Customer Projects - Americas
Company: Discourse
Posted: Not provided
Link: https://remotive.com/remote-jobs/software-dev/software-

In [ ]:
def scrapeFilter(title):
    url = f"https://remoteok.com/" 
    driver.get(url) 
    wait = WebDriverWait(driver, 10)  
    
    search_input = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="jobsboard"]/thead/tr/th/div[2]/input')))
    search_input.click()  
    filters_modal = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[6]/div/div[4]')))
    
    filter_items = filters_modal.find_elements(By.XPATH, './/div') 
    print(f"Number of filters found: {len(filter_items)}")
    filters =[]

    for filter in filter_items:
        filters.append(filter.text)
        
    return filters

scrapeFilter('Engineer')


Number of filters found: 100


['🤓 Engineer',
 '💼 Executive',
 '👵 Senior',
 '🤓 Developer',
 '💰 Finance',
 '♾️ Sys Admin',
 '☕️ JavaScript',
 '🍑 Backend',
 '🐀 Golang',
 '☁️ Cloud',
 '🚑 Medical',
 '🎨 Front End',
 '🥞 Full Stack',
 '♾️ Ops',
 '🎨 Design',
 '⚛️ React',
 '🔑 InfoSec',
 '🚥 Marketing',
 '📱 Mobile',
 '✍️ Content Writing',
 '📦 SaaS',
 '🎯 Recruiter',
 '⏰ Full Time',
 '🤖 API',
 '💼 Sales',
 '💎 Ruby',
 '👨\u200d🏫 Education',
 '♾️ DevOps',
 '👩\u200d🔬 Stats',
 '🐍 Python',
 '🔗 Node',
 '🇬🇧 English',
 '🔌 Non Tech',
 '📼 Video',
 '🎒 Travel',
 '🔬 Quality Assurance',
 '🛍 Ecommerce',
 '👨\u200d🏫 Teaching',
 '🐧 Linux',
 '☕️ Java',
 '🏅 Crypto',
 '👶 Junior',
 '📦 Git',
 '👩\u200d⚖️ Legal',
 '🤖 Android',
 '💼 Accounting',
 '♾️ Admin',
 '🖼 Microsoft',
 '📗 Excel',
 '🐘 PHP',
 '☁️ Amazon',
 '☁️ Serverless',
 '🎨 CSS',
 '🤓 Software',
 '🤔 Analyst',
 '🅰️ Angular',
 '🍏 iOS',
 '🎧 Customer Support',
 '🔡 HTML',
 '☁️ Salesforce',
 '🚥 Ads',
 '📦 Product Designer',
 '👋 HR',
 '🐬 SQL',
 '🔷 C',
 '🤓 Web Developer',
 '🚫 NoSQL',
 '🐬 Postgres',
 '➕ C Plus 

In [ ]:
def remoteokJobs(title):
    url = f"https://remoteok.com/remote-{title}-jobs" 
    driver.get(url) 
    wait = WebDriverWait(driver, 10)  
    try: 
        wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="jobsboard"]/tbody')))
        job_items = driver.find_elements(By.XPATH, '//*[@id="jobsboard"]/tbody/tr')
        job_items = [job for job in job_items if "job" in job.get_attribute('class')]
        print(f"Total jobs found: {len(job_items)}") 
        jobs = [] 
        for job in job_items:  
            try: 
                link_element = job.find_element(By.XPATH, './td[1]/a')  
                href = link_element.get_attribute('href') if link_element else "No link"
                title = job.find_element(By.TAG_NAME,'h2').text if job.find_elements(By.TAG_NAME,'h2') else "Unknown"
                company = job.find_element(By.TAG_NAME,'h3').text if job.find_elements(By.TAG_NAME,'h3') else "Unknown"
                location_elements = job.find_elements(By.CLASS_NAME, 'location')
                location = location_elements[0].text if location_elements else "Unknown"
                if "💰" in location:  
                    location = "Specified in Job description"
                job_type = "Remote"
                posted_elements = job.find_elements(By.CLASS_NAME,'time')
                posted = posted_elements[0].text if posted_elements else "Not provided"
                img_elements = job.find_elements(By.TAG_NAME, 'img')
                img = img_elements[0].get_attribute('src') if img_elements else "No Image"
                job_as_JSON = { 
                    "title": title, 
                    "company": company, 
                    "location": location, 
                    "posted": posted, 
                    "href": href, 
                    "img": img,
                    "type": job_type,
                } 
                jobs.append(job_as_JSON) 
                print(f"Title: {title}\nCompany: {company}\nPosted: {posted}\nLink: {href}\n") 
            except Exception as e: 
                print(f"Error extracting job details: {e}") 
        with open("remoteok_jobs.json", "w", encoding="utf-8") as f: 
            json.dump(jobs, f, indent=4, ensure_ascii=False) 
    except Exception as e: 
        print("Job listings not found or error:", e)
remoteokJobs('Engineer')


Total jobs found: 20
Title: Senior Frontend Engineer Design Matcha
Company: 0x
Posted: 1d
Link: https://remoteok.com/remote-jobs/remote-senior-frontend-engineer-design-matcha-0x-1076248

Title: Senior Software Engineer Webflow Labs
Company: Webflow
Posted: 1d
Link: https://remoteok.com/remote-jobs/remote-senior-software-engineer-webflow-labs-webflow-1076247

Title: Junior Developer Relations Engineer
Company: Flow Foundation
Posted: 1d
Link: https://remoteok.com/remote-jobs/remote-junior-developer-relations-engineer-flow-foundation-1076184

Title: Lead Software Engineer Digital Adherence
Company: Dimagi
Posted: 1d
Link: https://remoteok.com/remote-jobs/remote-lead-software-engineer-digital-adherence-dimagi-1076052

Title: Principal Software Engineer Ecosystem
Company: Pulumi Corporation
Posted: 1d
Link: https://remoteok.com/remote-jobs/remote-principal-software-engineer-ecosystem-pulumi-corporation-1076051

Title: Principal Engineer Automation & Connectivity
Company: Workiva Inc.
Poste